<a href="https://colab.research.google.com/github/patrickvonplaten/notebooks/blob/master/Distil_Whisper_Benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Distil Whisper

Distil-Whisper is a distilled version of the Whisper model that is 6 times faster, 49% smaller,
and performs within 1% WER on out-of-distribution evaluation sets.

First, let's have a super-fast recap on Whisper. Whisper was proposed by OpenAI
in [*Robust Speech Recognition via Large-Scale Weak Supervision*](https://cdn.openai.com/papers/whisper.pdf)
and is based on an encoder-decoder architecture that was trained on >600,000 hours of audio data.

To transcribe speech, the audio is passed through the encoder once and then the decoder auto-regressively
generates the transcription. OpenAI's best Whisper checkpoint, named *Whisper-large-v2* has both 32 encoder and
decoder layers. 32 is quite a lot. Let's visualize the model.



![img](https://huggingface.co/datasets/patrickvonplaten/images_distil/resolve/main/whiser_arch_old.png)

$\mathbf{X}_{1:T}$ represents hereby the speech input that in run through the encoder (shown in green) exactly
once. The encoder ouputs, *a.k.a* `encoder_hidden_states`, $\mathbf{H}_{1:M}$ are then used to condition
every decoder layer.

Starting with $y_0$ the decoder (shown in orange) then auto-regressively generates all tokens of the
transcription. In the visiualization above, there are 5 decoder forward passes (one pass for each $\mathbf{P}(y_i | \mathbf{y}_{0: i - 1}),  \forall i$).

In practice however, the decoder is run up to 128 times (depending on the length of the transcription) which means
that there are much much more passes through the decoder then the encoder.

Now, how does Distil-Whisper look like?

![img](https://huggingface.co/datasets/patrickvonplaten/images_distil/resolve/main/distil_arch_old.png)

Just two decoder layers! That means to generate a transcription of 128 tokens, Distil-Whisper needs to run
only 256 decoder layer forward passes, while Whisper--large-v2 has to run 4096 forward passes.

## Benchmarking

Great, now that we understood why Distil-Whisper should be faster in theory, let's see if it holds true in practice.

To begin with, we install `transformers`, `accelerate`, and `datasets`.

In [1]:
# !pip install --upgrade transformers accelerate datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.7 MB/s eta 0:00:00


In [1]:
!pip install git+https://github.com/huggingface/transformers accelerate datasets

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-1nplx1mj
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-1nplx1mj
  Resolved https://github.com/huggingface/transformers to commit 8801861d2de1568e8ca8f81d96a7ddf3964f6373
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In addition, we will make use of Flash-Attention 2 as it can both save
a lot of memory and speed up large matmul operations.

In [2]:
!pip install flash-attn --no-build-isolation

To begin with, let's load a dataset that we will use for benchmarking.

In [3]:
from datasets import load_dataset

dataset = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")

We start by benchmarking [`Whisper-large-v2`](https://huggingface.co/openai/whisper-large-v2) to get a reference number.
Let's load it and make sure to cast it to `float16` as well as setting `use_flash_attention_2=True`.

In [4]:
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v2"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True, use_flash_attention_2=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

You are attempting to use Flash Attention 2.0 with a model initialized on CPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


Great! For the benchmark, we will only measure the generation time, so let's write a short
helper function that measures the time it takes.

In [13]:
import time

def generate_with_time(model, inputs):
    start_time = time.time()
    outputs = model.generate(**inputs)
    generation_time = time.time() - start_time
    return outputs, generation_time

This function will return both the decoded tokens as well as the time
it took to run the model.

We now iterate over the audio samples and sum up the generation time.

In [20]:
import tqdm
all_time = 0

for sample in tqdm.tqdm(dataset):
  audio = sample["audio"]
  inputs = processor(audio["array"], sampling_rate=audio["sampling_rate"], return_tensors="pt")
  inputs = inputs.to(device=device, dtype=torch.float16)

  output, gen_time = generate_with_time(model, inputs)
  all_time += gen_time
  print(processor.batch_decode(output, skip_special_tokens=True))

print(all_time)

  1%|▏         | 1/73 [00:00<01:10,  1.02it/s]

[' Mr. Quilter is the apostle of the middle classes and we are glad to welcome his gospel.']


  3%|▎         | 2/73 [00:01<01:00,  1.18it/s]

[" Nor is Mr. Quilter's manner less interesting than his matter."]


  4%|▍         | 3/73 [00:03<01:20,  1.15s/it]

...

[" Breon saw something close to panic on his opponent's face when the man finally recognized his error."]


 99%|█████████▊| 72/73 [01:09<00:00,  1.05it/s]

[' A wave of despair rolled out from Irolg. Brienne sensed it and knew the fifth point was his.']


100%|██████████| 73/73 [01:09<00:00,  1.05it/s]

[" Then the powerful twist that's rested aside, in and under the guard."]
62.970786571502686


Alright! In total it took roughly 63 seconds to transcribe 73 audio samples.

Next, let's see how much time it takes with [Distil-Whisper](https://huggingface.co/distil-whisper/distil-large-v2).

In [21]:
# model_id = "distil-whisper/distil-large-v2"
model_id = "patrickvonplaten/whisper-large-v2-32-2"

distil_model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True, use_flash_attention_2=True
)
distil_model = distil_model.to(device)

You are attempting to use Flash Attention 2.0 with a model initialized on CPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


We run the same benchmarking loop.

In [22]:
import tqdm
all_time = 0

for sample in tqdm.tqdm(dataset):
  audio = sample["audio"]
  inputs = processor(audio["array"], sampling_rate=audio["sampling_rate"], return_tensors="pt")
  inputs = inputs.to(device=device, dtype=torch.float16)

  output, gen_time = generate_with_time(distil_model, inputs)
  all_time += gen_time
  print(processor.batch_decode(output, skip_special_tokens=True))

print(all_time)

  3%|▎         | 2/73 [00:00<00:13,  5.17it/s]

[' Mr. Quilter is the apostle of the middle classes, and we are glad to welcome his gospel.']
[" Nor is Mr. Quilter's manner less interesting than his matter."]


  4%|▍         | 3/73 [00:00<00:16,  4.20it/s]

[' He tells us that at this festive season of the year, with Christmas and roast beef looming before us, similes drawn from eating and its results occur most readily to the mind.']


  5%|▌         | 4/73 [00:00<00:16,  4.11it/s]

...

[" Brian saw something close to panic on his opponent's face when the man finally recognized his error."]


 99%|█████████▊| 72/73 [00:16<00:00,  4.50it/s]

[' A wave of despair rolled out from Irog. Brian sensed it and knew the fifth point was his.']


100%|██████████| 73/73 [00:16<00:00,  4.36it/s]

[" Then the powerful twist that's rested aside, in and under the guard."]
10.020044803619385


Only 10 seconds. A 6x speed-up!

## Memory

In addition to being significatnly faster, Distil-Whisper also has fewer parameters. Let's have a look
at how much fewer exactly.

In [18]:
distil_model.num_parameters() / model.num_parameters() * 100

49.000047275167184

Distil-Whisper is 49% of the size of Whisper. Note that this ratio is much lower if we would just compare the decoder.

In [19]:
distil_model.model.decoder.num_parameters() / model.model.decoder.num_parameters() * 100


13.175161920253482

As expected the decoder is much smaller. One might have guessed that it should even be less - something like 2/32, but we can't forget that the decoder has a very large word embedding that takes up a lot of space.

## Next steps

Hopefully this notebook shed some light on the motivation behind Distil-Whisper! For now, we've measured Distil-Whisper mainly on GPU, but are now actively looking into collaborating
to release code how to effectively accelerate Distil-Whisper on CPU as well.

Another very cool application of Distil-Whisper is speculative decoding. In speculative decoding,
we can use Distil-Whisper as a student model of Whisper-large-v2 to reach a speed-up of 2x without **any** loss in performance. More on that in a follow-up notebook that will come out soon!